In [7]:
#Installation blk.
!pip install mysql-connector-python
!pip install sshtunnel

In [15]:
#Import blk.
from   io         import StringIO
import pandas     as pd
import numpy      as np

import re
import mysql.connector
from   mysql.connector import Error

from   sshtunnel import SSHTunnelForwarder


In [22]:
#No IP4 blk

bastion_host   = "54.236.94.144"
bastion_user   = "ec2-user"
bastion_passwd = "123456"
rds_endpoint   = "tunnel-end-db.cdqeagyemzld.us-east-1.rds.amazonaws.com"
rds_port   = 1770
local_port = 0 #could be everything

# Establishing a tunnel; cuz we wanna a path: this Notebook -> EC2 -> RDS
tunnel = SSHTunnelForwarder(
    (bastion_host, 0),  # If you aint sure 'bout this port, use 0. Default is 22.
    ssh_username = bastion_user,
    ssh_password = bastion_passwd,
    remote_bind_address=(rds_endpoint, rds_port),
    local_bind_address=("localhost", local_port),
    allow_agent=False,
    #set_keepalive=10.0
)

tunnel.start()

try:
  conn = mysql.connector.connect(
        host     = "localhost",
        user     = "walkk",
        password = bastion_passwd,
        #database   = "dealercustomers",
        port     = tunnel.local_bind_port
    )

  cursor = conn.cursor()
  cursor.execute("SHOW DATABASES")

  #conn.commit()
  for basis in cursor.fetchall():
    print("\nLAODING...\n")
    print("Fetched: ", basis[0] ,"\n")


  conn.close()
  print( "Successfully fetched and closed RDS. \n")
except Error as e:
  print(f"Error: {e}")

print("Successfully connected and closed EC2. \n")
tunnel.close()


LAODING...

Fetched:  information_schema 


LAODING...

Fetched:  mysql 


LAODING...

Fetched:  performance_schema 


LAODING...

Fetched:  sys 

Successfully fetched and closed RDS. 

Successfully connected and closed EC2. 



#The whole implementation is commensurate with this in bash:

``ssh user-name@xxx.xxx.xxx.xxx -h something.rds.amazonaws.com -L -N -v``